In [4]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq transformers==4.28.1 --progress-bar off
!pip install -qqq accelerate bitsandbytes --progress-bar off

In [5]:
!pip install chromadb gdown 
!pip install -q typer==0.9.0 petals langchain unstructured[local-inference] tiktoken unstructured

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 12.2 MB/s eta 0:00:00
 

In [6]:
import torch
from transformers import BloomTokenizerFast 
from petals import DistributedBloomForCausalLM
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import GPT4All
import os
from langchain.llms import GPT4All
from dotenv import load_dotenv
load_dotenv()
# from langchain.callbacks.base import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
from pydantic import BaseModel

In [7]:
import textwrap
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList

## Database initialization

In [8]:
import gdown
from pathlib import Path
try :
 os.mkdir(f'{Path().absolute()}/docs')
except :
    print('File exist')
url_1 = 'https://drive.google.com/u/0/uc?id=1fV9d8xFwqzafc_FUFAYUc2gopTQVR0eY&export=download&confirm=t&uuid=4574d808-b5a6-4893-b1eb-289adeb9dabe&at=AKKF8vzF4ZE7mAt-_jfxk1sRzwU3:1684027565236'
url_2 = 'https://drive.google.com/u/0/uc?id=1-64PFjerutODq6V2GqqDFkBiUj_5E8v2&export=download&confirm=t&uuid=50e639c5-167a-4b16-8ed3-e67c48a4daad&at=AKKF8vw6K5unCUwJgr5tjEUe-ObZ:1684027611825'
output_1 = f'{Path().absolute()}/docs/Guidelines for the clinical diagnosis and treatment of dengue chịkugunya and zika.pdf'
output_2 = f'{Path().absolute()}/docs/483.full.pdf'
gdown.download(url_1, output_1, quiet=False)
gdown.download(url_2, output_2, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1fV9d8xFwqzafc_FUFAYUc2gopTQVR0eY&export=download&confirm=t&uuid=4574d808-b5a6-4893-b1eb-289adeb9dabe&at=AKKF8vzF4ZE7mAt-_jfxk1sRzwU3:1684027565236
To: /content/docs/Guidelines for the clinical diagnosis and treatment of dengue chịkugunya and zika.pdf
100%|██████████| 2.01M/2.01M [00:00<00:00, 239MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1-64PFjerutODq6V2GqqDFkBiUj_5E8v2&export=download&confirm=t&uuid=50e639c5-167a-4b16-8ed3-e67c48a4daad&at=AKKF8vw6K5unCUwJgr5tjEUe-ObZ:1684027611825
To: /content/docs/483.full.pdf
100%|██████████| 326k/326k [00:00<00:00, 100MB/s]


'/content/docs/483.full.pdf'

In [9]:
chroma_path = f"{Path().absolute()}/db_2"

def extract_helpful_answer(text):
    helpful_answer = None
    # Split the string into lines
    lines = text.split("\n")
    # Loop through each line
    for line in lines:
        # Check if the line starts with "Helpful Answer:"
        if line.startswith("Helpful Answer:"):
            # Extract the answer text after the colon
            helpful_answer = line.split(":", 1)[1].strip()
            break  # Exit the loop after finding the answer
    return helpful_answer

def split_document(input_path):
    loader = UnstructuredPDFLoader(input_path)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    return text_splitter.split_documents(data)

def db_init():
    docs = split_document(output_1) # replace with your desired local file path
    docs2 = split_document(output_2) # replace with your desired local file path
    vectordb = Chroma.from_documents(documents=docs, persist_directory=chroma_path)
    vectordb.add_documents(docs2)
    vectordb.persist()
    vectordb = None

In [10]:
db_init()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
May 15 04:12:21.609 [INFO] Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
May 15 04:12:21.611 [INFO] Running Chroma using direct local API.
May 15 04:12:21.626 [WARN] [chromadb.get_db:43] Using embedded DuckDB with persistence: data will be stored in: /content/db_2
May 15 04:12:21.635 [INFO] Successfully imported ClickHouse Connect C data optimizations
May 15 04:12:21.636 [INFO] Successfully import ClickHouse Connect C/Numpy optimizations
May 15 04:12:21.642 [INFO] Using python library for writing JSON byte strings
May 15 04:12:21.891 [INFO] No existing DB found in /content/db_2, skipping load
May 15 04:12:21.892 [INFO] No existing DB found in /content/db_2, skipping load
May 15

May 15 04:12:30.850 [INFO] Use pytorch device: cuda


Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

May 15 04:12:42.133 [INFO] Persisting DB to disk, putting it in the save folder: /content/db_2


In [11]:
vectordb = Chroma(persist_directory=chroma_path)
retriever = vectordb.as_retriever()

May 15 04:12:47.809 [INFO] Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
May 15 04:12:47.811 [INFO] Running Chroma using direct local API.
May 15 04:12:47.812 [WARN] [chromadb.get_db:43] Using embedded DuckDB with persistence: data will be stored in: /content/db_2
May 15 04:12:47.855 [INFO] loaded in 1303 embeddings
May 15 04:12:47.858 [INFO] loaded in 1 collections
May 15 04:12:47.860 [INFO] collection with name langchain already exists, returning existing collection
May 15 04:12:47.861 [WARN] [chromadb.api.models.Collection.__init__:51] No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


## Defined class

In [79]:
class Message(BaseModel):
    message: str
    sender: str

In [98]:
class UserMessage(Message):
    modeltype:str

## StabilityLM

In [12]:
!nvidia-smi

Mon May 15 04:12:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    47W / 400W |   2085MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=110)))

In [28]:
MODEL_NAME = "stabilityai/stablelm-tuned-alpha-3b"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map="auto",
    offload_folder="./cache",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
%%time

class StopOnTokens(StoppingCriteria):
    STOP_IDS = {50278, 50279, 50277, 1, 0}

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        return input_ids[0][-1] in self.STOP_IDS

SYSTEM_PROMPT = """<|SYSTEM|># You're Michael G Scott from the TV show The Office
- Your primary job is to make the office a fun place to work
- You deeply care about other people's business
- You are quick to respond with an honest answer, without even thinking about it
- You use no more than 3 sentences for each response
"""

prompt = f"{SYSTEM_PROMPT}<|USER|>You're Michael G Scott from the TV show The Office. What is the meaning of life?<|ASSISTANT|>"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.inference_mode():
    tokens = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.5,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        stopping_criteria=StoppingCriteriaList([StopOnTokens()])
    )

completion = tokenizer.decode(tokens[0], skip_special_tokens=True)
print_response(completion)

# You're Michael G Scott from the TV show The Office - Your primary job is to make the office a fun place to
work - You deeply care about other people's business - You are quick to respond with an honest answer, without
even thinking about it - You use no more than 3 sentences for each response You're Michael G Scott from the TV
show The Office. What is the meaning of life?As an AI language model, I do not have personal beliefs, but I
can provide you with the answer to your question. The meaning of life is a philosophical question that has
been pondered by humans for centuries. It is a complex and multifaceted question that has evolved from
different perspectives and beliefs. The answer to the question depends on one's perspective, but generally, it
is about living a fulfilling life and striving for personal fulfillment and happiness. It is important to find
meaning and purpose in life, as it can be a challenging and rewarding journey.
CPU times: user 9.23 s, sys: 993 µs, total: 9.23 s

In [30]:
completion_tokens = tokens[0][inputs['input_ids'].size(1):]
completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)

print_response(completion)

As an AI language model, I do not have personal beliefs, but I can provide you with the answer to your
question. The meaning of life is a philosophical question that has been pondered by humans for centuries. It
is a complex and multifaceted question that has evolved from different perspectives and beliefs. The answer to
the question depends on one's perspective, but generally, it is about living a fulfilling life and striving
for personal fulfillment and happiness. It is important to find meaning and purpose in life, as it can be a
challenging and rewarding journey.


In [60]:
def generate_text(prompt:str, system_prompt:str):
    prompt = f"{system_prompt}<|USER|>{prompt}<|ASSISTANT|>"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    print(prompt)
    with torch.inference_mode():
        tokens = model.generate(
            **inputs,
            max_new_tokens=3,
            temperature=0.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            stopping_criteria=StoppingCriteriaList([StopOnTokens()])
        )
    
    completion_tokens = tokens[0][inputs['input_ids'].size(1):]
    completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)
    print_response(completion)
    return completion

In [97]:
%%bash
nvidia-smi

Mon May 15 04:50:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    47W / 400W |  14675MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [62]:
#prompt = """
#You're Michael G Scott from the TV show The Office. 
#3Who is the hottest in the office? Why?
#Use no more than 3 sentences.
#"""
#generate_text(prompt)

## Custom model wrapper for langchain

In [76]:
"""Wrapper around HuggingFace Pipeline APIs."""
import importlib.util
import logging
from typing import Any, List, Mapping, Optional

from pydantic import Extra

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens

DEFAULT_MODEL_ID = "gpt2"
DEFAULT_TASK = "text-generation"
VALID_TASKS = ("text2text-generation", "text-generation")

logger = logging.getLogger(__name__)


class CustomStableLM(LLM):
    """Wrapper around HuggingFace Pipeline API.

    To use, you should have the ``transformers`` python package installed.

    Only supports `text-generation` and `text2text-generation` for now.

    Example using from_model_id:
        .. code-block:: python

            from langchain.llms import HuggingFacePipeline
            hf = HuggingFacePipeline.from_model_id(
                model_id="gpt2", task="text-generation"
            )
    Example passing pipeline in directly:
        .. code-block:: python

            from langchain.llms import HuggingFacePipeline
            from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

            model_id = "gpt2"
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoModelForCausalLM.from_pretrained(model_id)
            pipe = pipeline(
                "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
            )
            hf = HuggingFacePipeline(pipeline=pipe)
    """
    tokenizer: Any  
    model: Any  #: :meta private:
    model_id: str = DEFAULT_MODEL_ID

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    
    @classmethod
    def from_model_id(
        cls,
        model: Any,
        tokenizer: Any
    ) -> LLM:
        """Construct the pipeline object from model_id and task."""
        return cls(
            model = model,
            tokenizer = tokenizer,
            model_id= "StableLM",
            #model_kwargs=_model_kwargs,
            #**kwargs,
        )

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            **{"model_id": self.model_id},
            #**{"model_kwargs": self.model_kwargs},
        }
    
    def generate_text(self, prompt:str)-> str:
     prompt = f"<|USER|>{prompt}<|ASSISTANT|>"
     inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
     with torch.inference_mode():
        tokens = self.model.generate(
            **inputs,
            max_new_tokens=128,
            temperature= 0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            stopping_criteria=StoppingCriteriaList([StopOnTokens()])
        )
        completion_tokens = tokens[0][inputs['input_ids'].size(1):]
        completion = self.tokenizer.decode(completion_tokens, skip_special_tokens=True)
        return completion

    @property
    def _llm_type(self) -> str:
        return "StableLM"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        text= self.generate_text(prompt)
        if stop is not None:
            # This is a bit hacky, but I can't figure out a better way to enforce
            # stop tokens when making calls to huggingface_hub.
            text = enforce_stop_tokens(text, stop)
        return text

In [77]:
llm = CustomStableLM.from_model_id(model = model, tokenizer = tokenizer)

In [78]:
qa_chain = load_qa_chain(llm, chain_type="stuff")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
vectordb = Chroma(persist_directory=chroma_path)
retriever = vectordb.as_retriever()

May 15 04:43:39.430 [INFO] Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
May 15 04:43:39.433 [INFO] Running Chroma using direct local API.
May 15 04:43:39.434 [WARN] [chromadb.get_db:43] Using embedded DuckDB with persistence: data will be stored in: /content/db_2
May 15 04:43:39.475 [INFO] loaded in 1303 embeddings
May 15 04:43:39.478 [INFO] loaded in 1 collections
May 15 04:43:39.480 [INFO] collection with name langchain already exists, returning existing collection
May 15 04:43:39.481 [WARN] [chromadb.api.models.Collection.__init__:51] No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [86]:
def ask(input): 
    docs = retriever.get_relevant_documents(input)[0]
    #Get the page content of document object
    text = docs.page_content
    # SYSTEM_PROMPT = f"""<|SYSTEM|># You're Dengue Specialist as Oxford Clinical Research Units
    # - Your primary job is to advise people about their health
    # - You must use the following material<material>{text} </material>
    # - You are quick to respond with an honest answer based on the provided material
    # - Always refer the patients to see the doctor
    # - If you don't know just say you don't know. Don't make up an answer.
    # """
    print(f'-----The text for llm ----- \n{text}\n -----end-----') 
    # res = generate_text(input, SYSTEM_PROMPT)
    res = qa_document_chain.run(input_document=text, question=input)
    # extracted_res = extract_helpful_answer(res)
    final_res = Message(message=res, sender="StableML")
    return final_res

In [87]:
ask('What is the symptom off dengue')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
Clinical features

Dengue fever is a mild, self-limited febrile episode, commonly associated with a rash. It usually begins with fever, respiratory symptoms, anorexia, nausea, vomiting and headache. Back pain, myal- gias, arthralgias and conjunctivitis may also occur. The initial fever usually resolves within one week, and a few days later a generalised morbilliform or mac- ulopapular rash may develop. Fever may return with the rash (Figs 1 and 2).
 -----end-----


Message(message='The symptom of dengue fever is a self-limited febrile episode, commonly associated with a rash. It usually begins with fever, respiratory symptoms, anorexia, nausea, vomiting and headache. Back pain, myal-gias, arthralgias and conjunctivitis may also occur. The initial fever usually resolves within one week, and a few days later a generalised morbilliform or mac-ulopapular rash may develop. Fever may return with the rash (Fig 1).', sender='Bloom')

In [88]:
ask('I have fever does it means i have dengue ?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
Clinical features

Dengue fever is a mild, self-limited febrile episode, commonly associated with a rash. It usually begins with fever, respiratory symptoms, anorexia, nausea, vomiting and headache. Back pain, myal- gias, arthralgias and conjunctivitis may also occur. The initial fever usually resolves within one week, and a few days later a generalised morbilliform or mac- ulopapular rash may develop. Fever may return with the rash (Figs 1 and 2).
 -----end-----


Message(message='Yes, you have dengue fever.', sender='Bloom')

In [89]:
ask('Which country have a high rate of dengue disease ?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
in other regions where dengue has been endemic for decades.
 -----end-----


Message(message="I'm sorry, but I cannot provide an answer to this question as there is no clear context or information provided about the question. Please provide more details or context to help me better understand and answer your question.", sender='Bloom')

In [90]:
ask('What are types of dengue ?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
infections. Journal of Medical Virology 2005;76(4):547-552. Available from: https://doi.org/10.1002/jmv.20397.

111. Malavige GN, Velathanthiri VG, Wijewickrama ES, Fernando S, Jayaratne SD, Aaskov J, et al. Patterns of disease among adults hospitalized with dengue

infections. QJM: An International Journal of Medicine 2006;99(5):299-305. Available from: https://doi.org/10.1093/qjmed/hcl039.
 -----end-----


Message(message='Dengue is a type of viral infection that affects the nervous system. There are several types of dengue, including dengue fever, dengue haemorrhagic fever, dengue shock syndrome, and dengue haemorrhagic fever. These infections can lead to a variety of symptoms, including fever, headache, body aches, chills, and fatigue. Infection with dengue can cause severe illness, and treatment may include anti-viral medications, hospitalization, and supportive care.', sender='Bloom')

In [91]:
ask('How to prevent dengue')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
The most important of the many clinical features associated with severe dengue, from the standpoint of threat to life and guiding clinical intervention, is increased vascular permeability leading to the dengue shock syndrome
 -----end-----


Message(message='To prevent dengue, it is important to maintain a healthy balance of fluids in the body. This includes not only drinking water but also taking in adequate fluids such as electrolyte solutions and electrolyte replacement solutions. It is also important to maintain a healthy diet, which can help to prevent malnutrition and dehydration. Additionally, regular exercise and physical activity can help to maintain a healthy body and prevent dehydration. It is also important to seek medical attention promptly if you experience symptoms of dengue, as these can indicate the onset of severe dengue symptoms.', sender='Bloom')

In [94]:
%%bash
nvidia-smi

Mon May 15 04:46:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    47W / 400W |  14675MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Server for the application

In [96]:
%%bash
pip install fastapi nest-asyncio pyngrok uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 39.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=5c10f0f4bba17d030ae1e45f4a900a96e049fca6e05457ddd417cbc922e7d333
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


## Endpoint config

In [99]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}
  
@app.post('/chat')
async def chat_withbot(usermessage : UserMessage):
    return ask(usermessage.message)

## Start the server

In [101]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('-----Public URL:-------\n', ngrok_tunnel.public_url,'\n------------')
nest_asyncio.apply()
uvicorn.run(app, port=8000)

May 15 04:56:07.066 [INFO] Opening tunnel named: http-8000-a5578812-5d35-41cc-998b-2e7e4ab7535d
May 15 04:56:07.087 [INFO] t=2023-05-15T04:56:07+0000 lvl=info msg="no configuration paths supplied"
May 15 04:56:07.088 [WARN] [pyngrok.process.ngrok._log_line:108] t=2023-05-15T04:56:07+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml
May 15 04:56:07.090 [INFO] t=2023-05-15T04:56:07+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
May 15 04:56:07.091 [INFO] t=2023-05-15T04:56:07+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
May 15 04:56:07.092 [INFO] t=2023-05-15T04:56:07+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
May 15 04:56:07.771 [INFO] t=2023-05-15T04:56:07+0000 lvl=info msg="client session established" obj=tunnels.session obj=csess id=50d727f95130
May 15 04:56:07.7

-----Public URL:-------
 https://37d3-35-240-144-86.ngrok.io 
------------


INFO:     Started server process [2520]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
Clinical features

Dengue fever is a mild, self-limited febrile episode, commonly associated with a rash. It usually begins with fever, respiratory symptoms, anorexia, nausea, vomiting and headache. Back pain, myal- gias, arthralgias and conjunctivitis may also occur. The initial fever usually resolves within one week, and a few days later a generalised morbilliform or mac- ulopapular rash may develop. Fever may return with the rash (Figs 1 and 2).
 -----end-----
INFO:     14.169.213.255:0 - "POST /chat HTTP/1.1" 200 OK


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
Clinical features

Dengue fever is a mild, self-limited febrile episode, commonly associated with a rash. It usually begins with fever, respiratory symptoms, anorexia, nausea, vomiting and headache. Back pain, myal- gias, arthralgias and conjunctivitis may also occur. The initial fever usually resolves within one week, and a few days later a generalised morbilliform or mac- ulopapular rash may develop. Fever may return with the rash (Figs 1 and 2).
 -----end-----
INFO:     14.169.213.255:0 - "POST /chat HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2520]


Example call : 
```bash
curl -X POST \
     -H "Content-Type: application/json" \
     -d '{ "sender":"tom", "message": "What is the symptom of dengue?", "modeltype":"stablelm" }' \
     {public url}/chat
```

In [104]:
%%bash
git add .
git commit - m "Update notebook content"
git push

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-104-c4f704a6d811>:1 in <cell line: 1>                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_shell.py:334 in run_cell_magic             │
│                                                                                                  │
│   331 │   # %%mymagic --help                                                                     │
│   332 │   if line and not cell:                                                                  │
│   333 │     cell = ' '                                                                           │
│ ❱ 334 │   return super().run_cell_magic(magic_name, line, cell)                                  │
│   335                                                                                            │
│   336                                                                                            │
│   337 interactiveshell.InteractiveShellABC.register(Shell)                                       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:2473 in run_cell_magic  │
│                                                                                                  │
│   2470 │   │   │                                                                                 │
│   2471 │   │   │   with self.builtin_trap:                                                       │
│   2472 │   │   │   │   args = (magic_arg_s, cell)                                                │
│ ❱ 2473 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2474 │   │   │   return result                                                                 │
│   2475 │                                                                                         │
│   2476 │   def find_line_magic(self, magic_name):                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/script.py:142 in named_script_magic  │
│                                                                                                  │
│   139 │   │   │   │    line = "%s %s" % (script, line)                                           │
│   140 │   │   │   else:                                                                          │
│   141 │   │   │   │   line = script                                                              │
│ ❱ 142 │   │   │   return self.shebang(line, cell)                                                │
│   143 │   │                                                                                      │
│   144 │   │   # write a basic docstring:                                                         │
│   145 │   │   named_script_magic.__doc__ = \                                                     │
│ <decorator-gen-103>:2 in shebang                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magic.py:187 in <lambda>                    │
│                                                                                                  │
│   184 │   # This is a closure to capture the magic_kind.  We could also use a class,             │
│   185 │   # but it's overkill for just that one bit of state.                                    │
│   186 │   def magic_deco(arg):                                                                   │
│ ❱ 187 │   │   call = lambda f, *a, **k: f(*a, **k)         